In [1]:
import covasim as cv
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
import sciris as sc
import numpy as np

Covasim 3.1.4 (2022-10-22) — © 2020-2022 by IDM


In [2]:
class uniform_masking(cv.Intervention):
  def __init__(self,mask_eff=None,maskprob_ub=None,maskprob_lb=None,*args,**kwargs):
    super().__init__(**kwargs)
    self.mask_eff    = mask_eff
    self.maskprob_ub = maskprob_ub
    self.maskprob_lb = maskprob_lb
    self.num_masking    = []
    self.t              = []
    return

  def initialize(self,sim):
    super().initialize() 
    self.pop = len(sim.people)
    return
  
  def apply(self,sim):
    ppl                = sim.people
    ppl.rel_sus        = ppl.rel_sus
    self.t.append(sim.t) 
    self.num_dead      = ppl.dead.sum()
    self.num_diagnosed = (ppl.diagnosed & ppl.infectious).sum()
    self.x             = self.num_dead + self.num_diagnosed
    self.p             = np.exp(0.001 + 100*(self.x/self.pop)-0.001*sim.t)
    self.p             = (self.p/(1+self.p))-0.5 
    self.p             = np.clip(self.p,self.maskprob_lb,self.maskprob_ub)
    self.masking       = np.random.binomial(1,p=self.p,size=self.pop)
    self.num_masking.append(np.sum(self.masking))
    ppl.rel_sus        = np.where(self.masking,ppl.rel_sus*self.mask_eff,ppl.rel_sus)

  def plot(self):
    
    plt.plot(self.t,self.num_masking)
    plt.xlabel('Day')
    plt.ylabel('# of Agents Masking')
    plt.title('Masking Over Time')
    plt.show()
    return

In [ ]:
tp = cv.test_prob(symp_prob=0.1,asymp_prob=0.001,symp_quar_prob=0.3,asymp_quar_prob=0.3,quar_policy='daily')
um = uniform_masking(mask_eff=0.6,maskprob_ub=0.75,maskprob_lb=0.00)

In [ ]:
sim = cv.Sim(n_days=90,interventions=[tp,um])
sim1 = cv.Sim(interventions=tp,label='no masking',n_days=90)
msim = cv.MultiSim([sim,sim1])
msim.run()
num_mask = msim.sims[0].get_intervention(uniform_masking)
num_mask.plot()
msim.plot()